# mosasaurus example
This notebook shows how to run mosasaurus to extract spectra from a sample dataset. In this example, there's a small sample dataset of raw LDSS3C images stored in the directory `/Users/zkbt/Cosmos/Data/mosaurusexample/data/ut140809`. These data come from a transit observation of WASP-94Ab, and contain one of every expected filetype for LDSS3C.

### instrument
First we create an `instrument` object for LDSS3C. This contains all the instrument-specific information mosasaurus needs to be aware of (how to calibrate CCD images, what information to pull out of headers, where to find wavelength calibration information, etc...). We can also use it to set up some of the basics for how we should conduct the extraction for this instrument (how big of a subarray to think about for each target, parameters for extraction apertures, is it worth trying to zap cosmic rays?).

In [1]:
%matplotlib auto

# create an instrument with the appropriate settings
from mosasaurus.instruments import LDSS3C
i = LDSS3C(grism='vph-all')

# set up the basic directory structure, where `data/` should be found
path = '/Users/zkbt/Cosmos/Data/mosaurusexample'
i.setupDirectories(path)

# set the extraction defaults
i.extractiondefaults['spatialsubarray'] = 200
i.extractiondefaults['narrowest'] = 4
i.extractiondefaults['widest'] = 20
i.extractiondefaults['numberofapertures'] = 5
i.extractiondefaults['zapcosmics'] = False

i.normFlatFilterSize = (1,400)
# print out a summary of this instrument
i.summarize()

Using matplotlib backend: MacOSX
       [ldss3c] Here is a brief summary of ldss3c.

                 {'_line': inf,
                  '_mute': False,
                  '_pithy': False,
                  '_prefix': '       [ldss3c] ',
                  'alignmentranges': {'$H\\alpha$': (6425, 6725),
                                      '$H\\beta$': (4750, 5050),
                                      '$H_2O$': (9200, 9700),
                                      '$O_2$ - A': (7500, 7800),
                                      '$O_2$ - B': (6750, 7050),
                                      'Ca triplet': (8450, 8750)},
                  'arclamps': ['He', 'Ne', 'Ar'],
                  'baseDirectory': '/Users/zkbt/Cosmos/Data/mosaurusexample',
                  'binning': 2,
                  'dataDirectory': '/Users/zkbt/Cosmos/Data/mosaurusexample/data/',
                  'databottom': 0,
                  'dataleft': 0,
                  'dataright': 512,
                  'datatop'

### target
Next, we create a `target` object for the star we were looking at. This is mostly to pull out the RA and Dec for calculating barycentric corrections to the observation times.

In [2]:
# create a target, pulling values from simbad
from mosasaurus.Target import Target
import astropy.units as u
t = Target(starname='WASP-94A', name='WASP-94Ab')
t.summarize()
t.star.summarize()

         [star] made <WASP-94A> from SIMBAD
       [target] Here is a brief summary of target.

                 {'_line': inf,
                  '_mute': False,
                  '_pithy': False,
                  '_prefix': '       [target] ',
                  'name': 'WASP-94Ab',
                  'nametag': 'target',
                  'star': <WASP-94A>,
                  'starname': 'WASP-94A'}

         [star] Here is a brief summary of star.

                 {'_line': inf,
                  '_mute': False,
                  '_pithy': False,
                  '_prefix': '         [star] ',
                  'attributes': {'V': 10.050000190734863, 'comment': b'F8'},
                  'icrs': <SkyCoord (ICRS): (ra, dec) in deg
                     (313.78311, -34.13555306)>,
                  'name': 'WASP-94A',
                  'nametag': 'star',
                  'pmdec': -46.046,
                  'pmra': 24.868,
                  'simbadname': b'WASP-94A',
                  

### night
We create a `night` object to store information related to this night of observations. It will be connected to a data directory. For this, the night of `ut140809`, it expects a group of FITS images to be stored inside the base directory in `data/ut140809`. One thing this a `night` can do is set up a log of all the files, pulling information from their FITS headers.

In [3]:
# create a night to analyze
from mosasaurus.Night import Night
n = Night('ut140809', instrument=i)
n.createNightlyLog(remake=False)

        [night] Creating a log of all files on ut140809.
        [night] /Users/zkbt/Cosmos/Data/mosaurusexample/data/ut140809 contains 53 files matching "ccd*c1.fits"
        [night] Loaded a log file from /Users/zkbt/Cosmos/Data/mosaurusexample/working/nightly_log_ut140809.txt.
        [night] Loaded a condensed summary from /Users/zkbt/Cosmos/Data/mosaurusexample/working/nightly_summary_ut140809.txt.
        [night] Creating a log of all files on ut140809.
        [night] /Users/zkbt/Cosmos/Data/mosaurusexample/data/ut140809 contains 53 files matching "ccd*c1.fits"
        [night] Loaded a log file from /Users/zkbt/Cosmos/Data/mosaurusexample/working/nightly_log_ut140809.txt.
        [night] Loaded a condensed summary from /Users/zkbt/Cosmos/Data/mosaurusexample/working/nightly_summary_ut140809.txt.


### observation
The core unit of a `mosasaurus` analysis is an `observation`, pointing to a specific target with a specific instrument on a specific night. An observation will need to set up text files that indicate which file prefixes are associated with which type of file needed for a reduction.

In [4]:

# create an observation
from mosasaurus.Observation import Observation
o = Observation(t, i, n)
o.setupFilePrefixes(science=['wasp94'], reference=['wasp94 thru mask'], flat=['flat'])


  [observation] loaded a list of [dark] files from /Users/zkbt/Cosmos/Data/mosaurusexample/working/ut140809_WASP-94Ab/files/filesfordark.txt
  [observation] loaded a list of [bias] files from /Users/zkbt/Cosmos/Data/mosaurusexample/working/ut140809_WASP-94Ab/files/filesforbias.txt
  [observation] loaded a list of [flat] files from /Users/zkbt/Cosmos/Data/mosaurusexample/working/ut140809_WASP-94Ab/files/filesforflat.txt
  [observation] loaded a list of [He] files from /Users/zkbt/Cosmos/Data/mosaurusexample/working/ut140809_WASP-94Ab/files/filesforHe.txt
  [observation] loaded a list of [Ne] files from /Users/zkbt/Cosmos/Data/mosaurusexample/working/ut140809_WASP-94Ab/files/filesforNe.txt
  [observation] loaded a list of [Ar] files from /Users/zkbt/Cosmos/Data/mosaurusexample/working/ut140809_WASP-94Ab/files/filesforAr.txt
  [observation] loaded a list of [science] files from /Users/zkbt/Cosmos/Data/mosaurusexample/working/ut140809_WASP-94Ab/files/filesforscience.txt
  [observation] loa

### reducer
The `reducer` object will go in and extract spectra from that observation. Frankly, I'm not entirely sure anymore why this is distinct from an `observation` -- maybe we should just move the (very few) features of the `reducer` into `observation`, so we can just say `o.reduce()` straight away?

In [ ]:
# create a reducer to analyze this observation
from mosasaurus.Reducer import Reducer
r = Reducer(o, visualize=False)
r.reduce()

      [reducer] the reducing mosasaurus is grabbing a fly spanker to analyze
                 [Observation of <Target WASP-94Ab> with <Spectrograph LDSS3C> on <Night ut140809>]
  [calibration] setting up calibrator
  [calibration] creating master images for dict_keys(['dark', 'bias', 'flat', 'He', 'Ne', 'Ar', 'science', 'reference'])
  [calibration] creating a stacked master image for dark
  [calibration] populating the master dark image
  [calibration] loaded dark from /Users/zkbt/Cosmos/Data/mosaurusexample/working/ut140809_WASP-94Ab/calibrations/master_dark.fits
  [calibration] creating a stacked master image for bias
  [calibration] populating the master bias image
  [calibration] loaded bias from /Users/zkbt/Cosmos/Data/mosaurusexample/working/ut140809_WASP-94Ab/calibrations/master_bias.fits
  [calibration] creating a stacked master image for flat
  [calibration] populating the master flat image
  [calibration] loaded flat from /Users/zkbt/Cosmos/Data/mosaurusexample/working/ut140

/Users/zkbt/anaconda3/envs/mosasaurus/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


        [loupe] setting up loupe
        [loupe] 
        [loupe] your self.options include:
        [loupe]    [q]uit
        [loupe]    move the [c]rosshair, and plot slicey along it
        [loupe]    nudge the crosshair [up]
        [loupe]    nudge the crosshair [down]
        [loupe]    nudge the crosshair [left]
        [loupe]    nudge the crosshair [right]
        [iplot] waiting for a key to be pressed and released
        [iplot] "q" pressed at 1472.9870833333337, 198.72416666666663
        [loupe] "q" was pressed
     [aperture] Did you like the NormalizedFlat for this stamp? [Y,n]
     [aperture] saved normalized flat to /Users/zkbt/Cosmos/Data/mosaurusexample/working/ut140809_WASP-94Ab/extraction_default/aperture_713_1062/normFlat_aperture_713_1062.pdf
     [aperture] saved normalized flat to calibration stamp /Users/zkbt/Cosmos/Data/mosaurusexample/working/ut140809_WASP-94Ab/extraction_default/aperture_713_1062/calibStamps_aperture_713_1062.npy
     [aperture] extracting

### cubes
This is where it starts getting particularly kludgy. First, we create an unshifted cube, with every spectrum resample onto a uniform wavelength grid (although with a rough accuracy for the wavelength calibration).

In [6]:
from mosasaurus.Cube import Cube

# create a cube, using 16 pixel apertures
c = Cube(o, width=16)

# define which is the target, and which are comparisons
c.setStars(target='aperture_713_1062', comparisons='aperture_753_1062')

# populate the spectra in the cube, and save it
c.populate(shift=False, max=None)
c.save()
c.imageCube(keys=['raw_counts'], stars=[c.target])

         [cube] setting the [target] star to aperture_713_1062
         [cube] setting the [comparisons] star(s) to ['aperture_753_1062']
         [cube] setting the [other] stars to [None]
         [cube] attempting to load previously saved cubes from...
         [cube] /Users/zkbt/Cosmos/Data/mosaurusexample/working/ut140809_WASP-94Ab/extraction_default/spectralCube_WASP-94Ab_ut140809_2stars_25spectra_16px_raw.npy
         [cube]   loading [cubes] from the saved cube structure
         [cube]   loading [squares] from the saved cube structure
         [cube]   loading [temporal] from the saved cube structure
         [cube]   loading [spectral] from the saved cube structure
         [cube]   loading [stellar] from the saved cube structure
         [cube]   loading [target] from the saved cube structure
         [cube]   loading [comparisons] from the saved cube structure
         [cube] attempting to save the cube of loaded, shifted, compiled spectra to /Users/zkbt/Cosmos/Data/mosauru

In [ ]:
# estimate the required shifts for each exposures
from mosasaurus.WavelengthRecalibrator import WavelengthRecalibrator
wr = WavelengthRecalibrator(c)

# fix up the wavelength calibration for each exposure
r.mask.setup()
r.mask.addWavelengthCalibration(shift=True)


# repopulate the cube
c.populate(shift=True, remake=True)
c.imageCube(keys=['raw_counts'], stars=[c.target])
c.save()

# make movie of the cube
c.movieCube(stride=1, remake=True)

[wavelengthrecalibrator] attempting to load previously saved shifts and stretches from /Users/zkbt/Cosmos/Data/mosaurusexample/working/ut140809_WASP-94Ab/extraction_default/spectralstretch.npy
[wavelengthrecalibrator]   loading [corrections] from the saved cube structure
[wavelengthrecalibrator] loaded shifts successfully
         [mask] Extraction centers were already defined; loading them from /Users/zkbt/Cosmos/Data/mosaurusexample/working/ut140809_WASP-94Ab/extraction_default/extractionCenters.txt
         [mask] extraction centers are
         [mask]    x=[753. 713.]
         [mask]    y=[1062. 1062.]
     [aperture] created a spectroscopic aperture at (753.0, 1062.0)
     [aperture] populating calibration stamps
     [aperture] loaded calibration stamps from /Users/zkbt/Cosmos/Data/mosaurusexample/working/ut140809_WASP-94Ab/extraction_default/aperture_753_1062/calibStamps_aperture_753_1062.npy
     [aperture] created a spectroscopic aperture at (713.0, 1062.0)
     [aperture] pop